# Testing fastCPH

In [50]:
import numpy

In [51]:
import sklearn

In [52]:
import survwrap

In [53]:
#X, y = survwrap.load_test_data()
#X.shape, y.shape

In [54]:
my_data='flchain'
my_data_df=survwrap.datasets.get_data(my_data)
seed=2312

In [55]:
X, y= my_data_df.get_X_y()
X.shape, y.shape

((6524, 8), (6524,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [56]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [57]:
X_train, X_test, y_train, y_test = survwrap.survival_train_test_split(X, y,rng_seed=2311)

In [58]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((4893, 8), (1631, 8))

In [59]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [60]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(1962, 1472, 490)

In [61]:
splitter = survwrap.survival_crossval_splitter(X_train,y_train,n_splits=3, n_repeats=2,rng_seed=2309)
print([ (survwrap.get_indicator(y_train[_[1]]).sum()) for _ in splitter])

[490, 491, 491, 490, 491, 491]


## check possible dimensionality reduction

In [62]:
from sklearn.decomposition import PCA

In [63]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

8

Only a modest dimensionality reduction is possible using PCA

In [64]:
## Stratified CV spliter for survival analysis

In [65]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [66]:
#fastcph 
import lassonet
from lassonet import LassoNetCoxRegressor

In [67]:
fastcph=LassoNetCoxRegressor(tie_approximation='breslow',
                                hidden_dims=(8,8),
    lambda_start=0.001,
    path_multiplier=1.02,
    backtrack=True,
    #gamma=0.5,
    #val_size=0.15,                        
    verbose=2,
    random_state=seed,
    torch_seed=seed,
    )
fastcph

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [68]:
#X_train[:5]

In [69]:
#y_swap
#from pandas import DataFrame
#y_df= DataFrame({'time': survwrap.get_time(y_train),'event':survwrap.get_indicator(y_train)})
yup=numpy.column_stack((survwrap.get_time(y_train).astype('float32'),
                        survwrap.get_indicator(y_train).astype('float32')))
print(type(yup))
#yup=numpy.array(list(zip(survwrap.get_indicator(y_train).astype(float),survwrap.get_time(y_train).astype(float))))
yup[:10]
X_train.shape, yup.shape
#numpy.ndarray(y_train[1], y_train[0])

<class 'numpy.ndarray'>


((4893, 8), (4893, 2))

In [70]:
fitted_fastcph = fastcph.fit(X_train,yup)
fitted_fastcph

epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.348657608032227
epoch: 24
loss: 8.34302043914795
epoch: 25
loss: 8.337440490722656
epoch: 26
loss: 8.331913948059082
epoch: 27
loss: 8.326433181762695
epoch: 28
loss: 8.321002960205078
epoch: 29
loss: 8.31561660766

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [75]:
assert fastcph == fitted_fastcph

In [76]:
fastcph.set_params()

LassoNetCoxRegressor(backtrack=True, device=device(type='cpu'),
                     hidden_dims=(8, 8), lambda_start=0.001, random_state=2312,
                     tie_approximation='breslow', torch_seed=2312, val_size=0,
                     verbose=2)

In [73]:
# MACHECAZZO!
#fastcph.predict(X_train)[:10], fastcph.predict(X_test)[:10]
#lassonet.plot_path(fastcph, fastcph.path, X_train, yup)

In [77]:
fastcph.path_[1].objective

7.556235576868057

In [78]:
#fitted_fastcph.path_.loss , fitted_fastcph.path_.lambda 

lambda_min = min(fitted_fastcph.path_, key=(lambda x: x.objective))
#lambda_min = min([(_.objective, _.lambda_)  for _ in fitted_fastcph.path_])
lambda_min 


HistoryItem(lambda_=0.001999889552662456, state_dict=None, objective=7.554867652909718, loss=7.551932334899902, val_objective=7.554862884538136, val_loss=7.55192756652832, regularization=1.4677400588989258, l2_regularization=tensor(6.6267), l2_regularization_skip=tensor(0.5985), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=10)

In [80]:
fastcph.set_params(lambda_seq=[lambda_min.lambda_],
                   random_state=seed,torch_seed=seed,
                   verbose=1)
refit_fastcph=fastcph.fit(X_train,yup)

epoch: 0
loss: 8.509246826171875
epoch: 1
loss: 8.501243591308594
epoch: 2
loss: 8.493334770202637
epoch: 3
loss: 8.485517501831055
epoch: 4
loss: 8.477792739868164
epoch: 5
loss: 8.470152854919434
epoch: 6
loss: 8.462599754333496
epoch: 7
loss: 8.4551362991333
epoch: 8
loss: 8.447758674621582
epoch: 9
loss: 8.440472602844238
epoch: 10
loss: 8.433272361755371
epoch: 11
loss: 8.426156044006348
epoch: 12
loss: 8.419123649597168
epoch: 13
loss: 8.41217041015625
epoch: 14
loss: 8.40530014038086
epoch: 15
loss: 8.398512840270996
epoch: 16
loss: 8.391807556152344
epoch: 17
loss: 8.385175704956055
epoch: 18
loss: 8.378621101379395
epoch: 19
loss: 8.372139930725098
epoch: 20
loss: 8.365907669067383
epoch: 21
loss: 8.360102653503418
epoch: 22
loss: 8.354352951049805
epoch: 23
loss: 8.348657608032227
epoch: 24
loss: 8.34302043914795
epoch: 25
loss: 8.337440490722656
epoch: 26
loss: 8.331913948059082
epoch: 27
loss: 8.326433181762695
epoch: 28
loss: 8.321002960205078
epoch: 29
loss: 8.31561660766

In [81]:
fastcph.predict(X_train)

array([[0.82628715],
       [0.27492297],
       [3.2277865 ],
       ...,
       [1.584062  ],
       [3.1775737 ],
       [1.2817144 ]], dtype=float32)

In [82]:
fastcph.score(X_train,yup)

0.7971334581495467

In [83]:
#y_swap
#from pandas import DataFrame
#y_df= DataFrame({'time': survwrap.get_time(y_train),'event':survwrap.get_indicator(y_train)})
yup_test=numpy.column_stack((survwrap.get_time(y_test).astype('float32'),
                        survwrap.get_indicator(y_test).astype('float32')))
print(type(yup_test))
X_test.shape, yup_test.shape
#numpy.ndarray(y_train[1], y_train[0])

<class 'numpy.ndarray'>


((1631, 8), (1631, 2))

In [84]:
fastcph.score(X_test,yup_test)

0.7860323741294167

## Test with CV-based lambda selection

In [87]:
cv_set=survwrap.survival_crossval_splitter(X_train,y_train,
                                           n_repeats=1,
                                          rng_seed=seed)
cv_set

<generator object _RepeatedSplits.split at 0x7f3d77612ac0>

In [88]:
fastcph.get_params()

{'M': 10,
 'backtrack': True,
 'batch_size': None,
 'class_weight': None,
 'device': device(type='cpu'),
 'dropout': 0,
 'gamma': 0.0,
 'gamma_skip': 0.0,
 'groups': None,
 'hidden_dims': (8, 8),
 'lambda_seq': [0.001999889552662456],
 'lambda_start': 0.001,
 'n_iters': (1000, 100),
 'optim': None,
 'path_multiplier': 1.02,
 'patience': (100, 10),
 'random_state': 2312,
 'tie_approximation': 'breslow',
 'tol': 0.99,
 'torch_seed': 2312,
 'val_size': 0,
 'verbose': 1}

In [94]:
fastcph_cv=lassonet.LassoNetCoxRegressorCV(cv=cv_set,
                                          hidden_dims=(8,8),
                                          tie_approximation='breslow',
                                            path_multiplier=1.02,
                                          torch_seed=seed,random_state=seed)
fastcph_cv

LassoNetCoxRegressorCV(cv=_CVIterableWrapper(cv=[]))

In [96]:
fastcph_cv.fit(X_train,yup)

Choosing lambda with cross-validation: 0it [00:00, ?it/s]


ValueError: min() arg is an empty sequence